In [1]:
# Import libraries:
import pandas as pd
import matplotlib.pyplot as plt
import dt_ids7_export_utils as bh_utils

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
else:
    path = 'G:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'IDS7 RRH XA.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH XA reduced.xlsx')

del path

# Check for missing time data:
df_ids7 = bh_utils.filter_NaT(df_ids7)
# Check for invalid accession numbers:
df_ids7 = bh_utils.check_accession_format(df_ids7, True)

PermissionError: [Errno 13] Permission denied: 'E:\\IDS7 RRH XA.xlsx'

1. Iterate through the Accession No. In the IDS7 data.
2. Check if there are more than one accession No. for the current patient at the current date and time.

3. Check if there are corresponding entries of accession No. in the dosetrack data. (DoseTrack should only have one Accession No).
4. Is a corresponding accession number os found in DoseTrack, use this accession No.
5. If there is no corresponding accession No. in DoseTrack, mark the case as having missing DoseTrack data.

8. Do a search in the DoseTrack data for duplicate dose Data. If there are lines with identical dose, remove the duplicates. There should be a minor dose difference in all CBCT acquisitions for instance.
9. Go through each accession No. in DoseTrack and Merge Plane A and B doses, and merge any additional CBCT examinations.

In [ ]:
# Go through the IDS7 data by accession No.
# Check if there are any other accession No. on the same patient with the same booking time.

def compare_accession(df_ids7, df_dt):
    acquisition_no = df_ids7['Henvisnings-ID'].drop_duplicates()
    
    # First we would like to check whether a patient has more than one accession number for a single booking. 


    
    status_acc_no = pd.Series(index=acc_no)
    for acc_no in status_acc_no.index:
        # find the patient ID and the time of the examination:
        patient_id = df_ids7[df_ids7['Henvisnings-ID'] == acc_no]['Pasient'].iloc[0]
        time = df_ids7[df_ids7['Henvisnings-ID'] == acc_no]['Bestilt dato og tidspunkt'].iloc[0]

        2+2



In [ ]:
# Check whether an accession number is in the DoseTrack data:
def check_accession_ids7_vs_dt(df_ids7, dfdt, verbose=False):
    # go thorugh the accession numbers in the IDS7 data and check whether they are in the DoseTrack data:
    acc_no = df_ids7['Henvisnings-ID'].drop_duplicates()
    status_acc_no = pd.Series(index=acc_no)
    for acc_no in status_acc_no.index:
        if acc_no in df_dt['Accession Number'].values:
            status_acc_no[acc_no] = True
        else:
            status_acc_no[acc_no] = False
            if verbose:
                print('Accession number {} not in DoseTrack data.'.format(acc_no))
    
    return status_acc_no

status = check_accession_ids7_vs_dt(df_ids7, df_dt, True)

In [11]:
# First we check what the lowest time interval between two bookings with different accession numbers is for the same patient:

patients = df_ids7['Pasient'].drop_duplicates()
# Create an pandas series to store the lowest time interval for each patient between two bookings with different accession numbers:
lowest_time_interval = pd.Series(index=patients)
for patient in patients:
    # Get the accession numbers for the patient:
    acc_no = df_ids7[df_ids7['Pasient'] == patient]['Henvisnings-ID'].drop_duplicates()
    # Get the time of the examination for the patient:
    time = df_ids7[df_ids7['Pasient'] == patient]['Bestilt dato og tidspunkt']
    # Sort the time:
    time = time.sort_values()
    # Calculate the time interval between two bookings in hours:
    time_interval = (time - time.shift()).dropna().dt.total_seconds() / 3600
        # Get the lowest time interval:
    lowest_time_interval[patient] = time_interval.min()


One patient can have the same accession no on different laboratories, due to erroneous registration in PACS.
If so please check the Modality/Room in DoseTrack.

If there is two different accession numbers at the same starting time they are likely the same examination.
There was an example of a patient with as little as 1 hr 22 min between examinations. 

If there is two different accession numbers with the exact same time they are likely the same examination. 

In [ ]:

acquisition_no = df_ids7['Henvisnings-ID'].drop_duplicates()
status_acc_no = pd.Series(index=acquisition_no)
for acc_no in status_acc_no.index:
    # find the patient ID and the time of the examination:
    patient_id = df_ids7[df_ids7['Henvisnings-ID'] == acc_no]['Pasient'].iloc[0]
    time = df_ids7[df_ids7['Henvisnings-ID'] == acc_no]['Bestilt dato og tidspunkt'].iloc[0]
    print(patient_id, time)

